### Dynamic Crawling and static extracting stock news from AAStocks http://www.aastocks.com/en/stocks/news/aafn-company-news

In [8]:
import re
import requests
from lxml import etree
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin
import json

import pandas as pd
#pd.set_option('display.max_columns', None)
import numpy as np
import numexpr as ne

!pip install selenium
!apt-get -y update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

#webdriver.Chrome().browser. find_elements(By.CLASS_NAME,'inline_block') all '%inline_block%' bars <div class="inline_block lastupdsep"><div>/ (By.CSS_SELECTOR,'p.inline_block')all exact bars <p class="inline_block">/ (By.PARTIAL_LINK_TEXT,'')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#browser.get("http://music.163.com")
#print(browser.page_source)
# browser.get(url)
# time.sleep(3)

#browser.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


**Headline | Releasing time | Polarity**

In [67]:
url = "http://www.aastocks.com/en/stocks/news/aafn-company-news"
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    "Referer": "http://www.aastocks.com/en/stocks/news/aafn/latest-news/0"
}
# How many times scrolls to bottum is required in initial url?
times = 1

#def scrape(url, times): #def scrapeWeeklyCompNews(url, times):

if not url.startswith('http'):
    raise Exception('URLs need to start with "http"')

browser = webdriver.Chrome('chromedriver',options=chrome_options)
print('Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)')
browser.get(url)
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 5 # load all weekly news (15 pages) # You can set your own pause time. My laptop is a bit slow so I use 5 sec
#screen_height = browser.execute_script("return window.screen.height;")   # get the screen height of the web
previous_height = browser.execute_script("return document.body.scrollHeight;")  # starting height
counter = 1

while True: # load all info in initial page
    # scroll one screen height each time
    browser.execute_script("window.scrollTo(0, {previous_height}*{i});".format(previous_height=previous_height, i=counter))
    #html = browser.find_element_by_tag_name('html') # Or direct send key
    #html.send_keys(Keys.END)
    time.sleep(scroll_pause_time) # load all weekly news (15 pages)

    # whens.extend(t.text for t in list(browser.find_elements(By.CSS_SELECTOR,'div.inline_block')) if len(t.text)==16)
    # print(len(whens),'whens inserted: [..,"'+whens[-1]+'"],',counter,'pages done')
    counter += 1

    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    # scroll_height = browser.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if counter > times:  #if (screen_height) * i > scroll_height:
      break

## get ['Releasing time'], ['Polarity']
# after loading all documents..
whens = [tve.text for i,tve in enumerate(browser.find_elements(By.CSS_SELECTOR,'div.inline_block')) if len(tve.text)==16]
print(len(whens),'whens inserted: [..,"'+whens[-1]+'"],',counter-1,'pages done')
pos = [] # pos = [int(tve.text.replace('Positive','')) for i,tve in enumerate(browser.find_elements(By.CLASS_NAME,'divBullish'))]# if i%2 == 0]# even i for Bullish ,odd i for Bearish
neg = [] # neg = [int(tve.text.replace('Negative','')) for i,tve in enumerate(browser.find_elements(By.CLASS_NAME,'divBearish'))]# if i%2 != 0]# even i for Bullish ,odd i for Bearish
#VE = list(set(pos)-set(neg))  # VE = eval(tve.find_all(class_='value')[1].text+'-'+tve.find_all(class_='value')[2].text)
Polarity = []
for tve in list(browser.find_elements(By.CLASS_NAME,'div_VoteTotal')): # even i divBullish ,odd i for Bearish

  Bullish = int(tve.text.split('tive')[1].replace('Nega',''))
  Bearish = int(tve.text.split('tive')[2])
  VE = Bullish-Bearish

  pos.append(Bullish)
  neg.append(Bearish)
  Polarity.append('positive' if VE > 0 else 'negative' if VE < 0 else 'neutral')
print(len(Polarity),'Polarity inserted: [..,"'+Polarity[-1]+'"],',counter-1,'pages done')

## get ['Headline'] & news follow links
headlines = [] #headlines = [h.text for h in list(browser.find_elements(By.CLASS_NAME,'newshead4'))]
links = []
for h in list(browser.find_elements(By.XPATH,"//div[contains(@class, 'newshead4')]/a")):
  headlines.append(h.text)
  links.append(h.get_attribute('href'))
print(len(headlines),'headlines inserted: [..,"'+headlines[-1]+'"],',counter-1,'pages done')
print(len(links),'links inserted: [..,"'+links[-1]+'"],',counter-1,'pages done')
print('Thank you for your patience :)')

# if __name__ == '__main__':
#     scrape(url, 1) # How many times scrolls to bottum is required in initial url?

Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)
300 whens inserted: [..,"2022/06/27 18:25"], 1 pages done
300 Polarity inserted: [..,"positive"], 1 pages done
300 headlines inserted: [..,"HKD1.9B Southbound Trading Net Inflow to HKEX"], 1 pages done
300 links inserted: [..,"http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1193462/company-news/HK6"], 1 pages done
Thank you for your patience :)


In [69]:
browser.close() # close initial page

In [68]:
links # check validity for next scrape

['http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195085/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195083/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195076/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195062/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195055/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195050/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195044/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195035/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195038/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195039/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195029/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195026/company-news/HK6',
 'http://www.aastocks.com/en

In [68]:
# concat all columns
df = pd.DataFrame({'Headline':headlines, 'Releasing time':whens, 'Polarity':Polarity, 'Positive':pos, 'Negative':neg})
df = df.convert_dtypes()
df

,Headline,Releasing time,Polarity,Positive,Negative
0,<IPO News>TUYA-W (02391.HK) Closes at $19.62 i...,2022/07/04 18:32,positive,5,2
1,HKD676M Southbound Trading Net Outflow from ME...,2022/07/04 18:25,positive,39,19
2,"MEITUAN Co-Founder Sells 166K Class B Shrs, Ca...",2022/07/04 17:22,negative,15,16
3,DAFA PPT 1H22 Contracted Sales Slip 85.6% YoY,2022/07/04 17:03,positive,2,0
4,GREATWALL MOTOR Repurchases 61.6475M H-shrs Cu...,2022/07/04 16:50,positive,39,8
...,...,...,...,...,...
295,SOUTH CHINA FIN Sells 600K CHINA EAST AIR Shrs...,2022/06/28 01:42,positive,11,2
296,<Result Ann>BAUHAUS INT'L Annual Net Profit $1...,2022/06/28 01:42,positive,12,5
297,ANALOGUE HLDGS (01977.HK) Estimates Interim NP...,2022/06/28 01:36,neutural,6,6
298,MO Covid Positive Cases Add to 391,2022/06/27 18:34,positive,38,29


In [54]:
df['Headline']

0      <IPO News>TUYA-W (02391.HK) Closes at $19.62 i...
1      HKD676M Southbound Trading Net Outflow from ME...
2      MEITUAN Co-Founder Sells 166K Class B Shrs, Ca...
3          DAFA PPT 1H22 Contracted Sales Slip 85.6% YoY
4      GREATWALL MOTOR Repurchases 61.6475M H-shrs Cu...
                             ...                        
295    SOUTH CHINA FIN Sells 600K CHINA EAST AIR Shrs...
296    <Result Ann>BAUHAUS INT'L Annual Net Profit $1...
297    ANALOGUE HLDGS (01977.HK) Estimates Interim NP...
298                   MO Covid Positive Cases Add to 391
299        HKD1.9B Southbound Trading Net Inflow to HKEX
Name: Headline, Length: 300, dtype: string

**Company name | Stock Code | Abstract** (loading and retreive requiering ~10+time.sleep(2) second per news, using links extracted from above)

In [54]:
fol_headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    "Referer": "http://www.aastocks.com/en/stocks/news/aafn-company-news"
    }
symbols = []
names = []
abstracts = []
browser = webdriver.Chrome('chromedriver',options=chrome_options)
print('Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)')
# 300 links requiring ~1hr (including 10 mins time.sleep), ~12 sec/link to retrieve [name], [symbol] & [abstract]
for link in links: # follow each news links to get [name], [symbol] & [abstract]
  #wait = WebDriverWait(browser, scroll_pause_time) # we don't want scroll initial page again https://stackoverflow.com/questions/70683534/python-selenium-iterate-table-of-links-clicking-each-link
  print(link)
  browser.get(link)
  print('Loading 2 sec to pretend as a human..')
  time.sleep(2)
  abstracts.append(browser.find_element(By.XPATH,'//p').text)
  print(len(abstracts),'abstracts inserted')
  #sym = browser.execute_script('return NewsRelativeSymbol') # str[:5]
  # sess = requests.session()
  # response = sess.get(link, headers=fol_headers)
  # soup = BeautifulSoup(response.json(),'lxml') #.content html.parser
  #abstract.append(soup.find("p").text) #.get_text(strip=True,separator='<br>').split('<br>')#.text.strip().split('\n')[0]#.strip()#.contents.strip()#class_= 'newscontent5 fLevel3')
  #sym = soup.find("a",class_='jsStock jsBmpStock').get('sym')
  try:
  #   sym = soup.find("a",class_='jsStock jsBmpStock')['sym']
    button = browser.find_element(By.XPATH,'//a[@class="jsStock jsBmpStock"]')#.get_attribute('href') # 1st relevant symbol appears

  except Exception as e:
    print(e,'\nNo Ticker button appears, getting Related Symbol')
    # json_string = re.search(r'NewsRelativeSymbol\s*=\s*(.*)', str(soup.find('script')), flags=re.DOTALL)
    # #sym = soup.find('a',class_='more-lnk')
    # sym = json.loads(json_string[1])
    # above return empty...
    button = browser.find_element(By.LINK_TEXT,"Related News") # related symbol from related news link

  browser.execute_script("arguments[0].click();", button)
  symbols.append(browser.find_element(By.XPATH,'//div[@id="SQ_Symbol"]').text[1:6])
  print(len(symbols),'symbols inserted: [..,"'+symbols[-1]+'"],')
  names.append(browser.find_element(By.XPATH,'//label[@id="SQ_Name"]').text)
  print(len(names),'company names inserted: [..,"'+names[-1]+'"]')


print('Thank you for your patience :)')

Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)
http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1195085/company-news/HK6
Loading 2 sec to pretend as a human..
1 abstracts inserted
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class="jsStock jsBmpStock"]"}
  (Session info: headless chrome=101.0.4951.64)
Stacktrace:
#0 0x562ea6b02553 <unknown>
#1 0x562ea67ed183 <unknown>
#2 0x562ea6823619 <unknown>
#3 0x562ea6823831 <unknown>
#4 0x562ea6858087 <unknown>
#5 0x562ea684101d <unknown>
#6 0x562ea6855d6c <unknown>
#7 0x562ea6841363 <unknown>
#8 0x562ea681747c <unknown>
#9 0x562ea6818945 <unknown>
#10 0x562ea6b729d0 <unknown>
#11 0x562ea6b37a38 <unknown>
#12 0x562ea6b3775c <unknown>
#13 0x562ea6b37fc2 <unknown>
#14 0x562ea6b7071b <unknown>
#15 0x562ea6b38221 <unknown>
#16 0x562ea6b1b5b3 <unknown>
#17 0x562ea6b41988 <unknown>
#18 0x562ea6b41b1a <unknown>
#19 0x562ea6b5aa41 <unknown>
#20 0x7fc9b61726db <unknown>
 

In [55]:
browser.close()

In [55]:
# after finish digging..
df['Company name'] = names
df['Stock Code'] = symbols
df['Abstract'] = abstracts

**Headline | Releasing time | Company name | Stock Code | Abstract | Polarity**

In [74]:
df = df[['Headline', 'Releasing time', 'Company name', 'Stock Code', 'Abstract', 'Polarity', 'Positive', 'Negative']]
df = df.convert_dtypes()
df['Releasing time'] = pd.to_datetime(df['Releasing time'])
df['Stock Code'] = df['Stock Code'].astype('category')
df['Polarity'] = df['Polarity'].astype('category')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Headline        300 non-null    string        
 1   Releasing time  300 non-null    datetime64[ns]
 2   Company name    300 non-null    string        
 3   Stock Code      300 non-null    category      
 4   Abstract        300 non-null    string        
 5   Polarity        300 non-null    category      
 6   Positive        300 non-null    Int64         
 7   Negative        300 non-null    Int64         
dtypes: Int64(2), category(2), datetime64[ns](1), string(3)
memory usage: 20.4 KB


,Headline,Releasing time,Company name,Stock Code,Abstract,Polarity,Positive,Negative
0,<IPO News>TUYA-W (02391.HK) Closes at $19.62 i...,2022-07-04 18:32:00,HKEX,00388,"TUYA-W (02391.HK), set to go public tomorrow, ...",positive,5,2
1,HKD676M Southbound Trading Net Outflow from ME...,2022-07-04 18:25:00,LI AUTO-W,02015,"There was HKD645.9 million, HKD341.9 million a...",positive,39,19
2,"MEITUAN Co-Founder Sells 166K Class B Shrs, Ca...",2022-07-04 17:22:00,MEITUAN-W,03690,MEITUAN-W (03690.HK) +7.000 (+3.605%) Shor...,negative,15,16
3,DAFA PPT 1H22 Contracted Sales Slip 85.6% YoY,2022-07-04 17:03:00,DAFA PPT,06111,DAFA PPT (06111.HK) -0.025 (-5.747%) announ...,positive,2,0
4,GREATWALL MOTOR Repurchases 61.6475M H-shrs Cu...,2022-07-04 16:50:00,GREATWALL MO,02333,GREATWALL MOTOR (02333.HK) -0.960 (-5.948%) ...,positive,39,8
...,...,...,...,...,...,...,...,...
295,SOUTH CHINA FIN Sells 600K CHINA EAST AIR Shrs...,2022-06-28 01:42:00,SOUTH CHINA,00619,SOUTH CHINA FIN (00619.HK) -0.005 (-1.695%) ...,positive,11,2
296,<Result Ann>BAUHAUS INT'L Annual Net Profit $1...,2022-06-28 01:42:00,BAUHAUS INT',00483,BAUHAUS INT'L (00483.HK) -0.020 (-2.222%) a...,positive,12,5
297,ANALOGUE HLDGS (01977.HK) Estimates Interim NP...,2022-06-28 01:36:00,ANALOGUE HLD,01977,ANALOGUE HLDGS (01977.HK) -0.030 (-2.308%) ...,neutural,6,6
298,MO Covid Positive Cases Add to 391,2022-06-27 18:34:00,GALAXY ENT,00027,Macau Health Bureau announced that the patient...,positive,38,29


In [97]:
df.to_excel(f'News Repository/AAstocksWEEKLYcompNews_{df['Releasing time'][-1].strftime("%m/%d/%Y")}-{df['Releasing time'][0].strftime("%m/%d/%Y")}.xlsx', index=True, header = df.columns) # .xlsx can fixed grid size